In [1]:
import openai
import os

In [5]:
# 定義 prompt
prompt_def = """
Please define the following categories:
1. Celebrity
2. Professor in college of social sciences, humanities, or education
3. Politician
4. Journalist/Media
5. General Public
6. Perpetrator
7. Victim
8. Business Owners
"""
openai.api_key = "YOUR_API_KEY"
response_def = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt_def}
    ]
)

print(response_def.choices[0].message["content"])


1. Celebrity: A person who is widely recognized in a society or by the general public, usually because of their work in the entertainment industry, including fields like film, music, television, sports or due to their association with such personalities. 

2. Professors in colleges of social sciences, humanities, or education: These are educators and researchers who work in tertiary education institutions. Their expertise lies in social sciences (such as sociology, anthropology, etc.), humanities (like literature, philosophy, etc.) or education (education theory, learning methodologies, etc.). They teach and publish research to expand knowledge in their respective fields.

3. Politician: An individual who is involved in influencing public policy and decision making. This includes people who hold office in government, people who seek to gain office through election, or those who have significant influence in the political landscape of a society.

4. Journalist/Media: A journalist is a p

In [8]:
# 假設 GPT 回答了某個定義，然後你在第二步的 prompt 中包含之前的對話
conversation_history = f"User: {prompt_def}\nAssistant: {response_def.choices[0].message['content']}"

# 第二步請求，基於上一步的對話
prompt_2 = """
Here are some individuals. For each, categorize them as either "celebrity", "professors in college of social sciences, humanities, or education", "politician", "journalist/media", "general public", "perpetrator", "victim", or "business owner".
Please do not use categories that are not defined in the previous response. If you do not know the answer, please say "unknown" and further explain the reason why you don't know.

1. Barack Obama
2. Taylor Swift
3. Anderson Cooper
4. A random person on the street
5. A guy killed a woman
6. A woman got murdered 
7. A professor in the department of psychology
8. A professor in the department of information management
9. A founder of a tech company

Provide the classification in the following format:
- Name: [Classification]
"""
prompt_test = f"{conversation_history}\nUser: {prompt_2}"
response_test = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt_test}
    ]
)
print(response_test.choices[0].message["content"])


- Barack Obama: Politician
- Taylor Swift: Celebrity
- Anderson Cooper: Journalist/Media
- A random person on the street: General Public
- A guy who killed a woman: Perpetrator
- A woman who got murdered: Victim
- A professor in the department of psychology: Professors in college of social sciences, humanities, or education
- A professor in the department of information management: Unknown (as 'Information Management' does not fall under previosly defined categories of social sciences, humanities, or education)
- A founder of a tech company: Business Owner


In [11]:
def classify_identities(article_text):
    prompt = f"""
    Analyze the following text and extract the names and their identities as either "celebrity", "professor in college of social sciences, humanities, or education", "politician", "journalist/media", "general public", "perpetrator", "victim", or "business owner" by definition provided. 

    Definition:
    {conversation_history}
    
    Text:
    {article_text}
    Also, you should determine whether the entity is "asian", "non-asian" or "unknown".
    
    Provide the result in the following JSON format: 
    {{
    "entities": [
        {{
            "name": "",
            "identity": ""
            "isasian": ""
        }},
        {{
            "name": "",
            "identity": ""
            "isasian": ""
        }}
    ]
    }}
    After completing the analysis, provide your JSON output within <result> tags
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message["content"]

In [12]:
valid_identities = ["celebrity", "professor in college of social sciences, humanities, or education", "politician", "journalist/media", "general public", "perpetrator", "victim", "business owner"]
valid_isasian = ["asian", "non-asian", "unknown"]
# 檢查 JSON 格式和 identity 是否有效
def validate_json_and_identity(json_data):
    try:        
        # 檢查是否存在 "entities" 字段，並且它是否是一個列表
        if "entities" in json_data and isinstance(json_data["entities"], list):
            # 檢查每個實體是否有 "name" 和 "identity" 字段，並且 identity 是否有效
            for entity in json_data["entities"]:
                if "name" not in entity or "identity" not in entity or "isasian" not in entity:
                    return False
                if entity["identity"].lower() not in valid_identities:
                    return False
                if entity["isasian"].lower() not in valid_isasian:
                    return False
            return True
        else:
            return False
    except json.JSONDecodeError:
        # 如果無法解析成 JSON，則返回 False
        return False

In [13]:
import json
articles_dir = "../../test" # 要正式跑的時候改成 codes/rag/articles
results = []

# 遍歷所有文章文件
for filename in os.listdir(articles_dir):
    rerun_times = 3
    if filename.endswith(".txt"):  
        with open(os.path.join(articles_dir, filename), "r", encoding="utf-8") as file:
            text = file.read()
            content = classify_identities(text)
            content = content.split("<result>")[1].split("</result>")[0].lower()
            content_json = json.loads(content)
            if validate_json_and_identity(content_json):
                print(f"{filename} success")
            while not validate_json_and_identity(content_json) and rerun_times>0:
                content = classify_identities(text)
                content = content.split("<result>")[1].split("</result>")[0].lower()
                content_json = json.loads(content)
                rerun_times = rerun_times - 1
                if validate_json_and_identity(content_json):
                    print(f"{filename} success")
            if not validate_json_and_identity(content_json) and rerun_times == 0:
                print(f"{filename} failed")
            results.append(
                content_json
            )
print(results)

5.txt success
4.txt success
3.txt failed
2.txt success
1.txt failed
[{'entities': [{'name': 'yin, alice', 'identity': 'journalist/media', 'isasian': 'asian'}, {'name': 'hong liu', 'identity': 'general public', 'isasian': 'asian'}, {'name': 'robert aaron long', 'identity': 'perpetrator', 'isasian': 'non-asian'}, {'name': 'byron sigcho-lopez', 'identity': 'politician', 'isasian': 'unknown'}, {'name': 'lori lightfoot', 'identity': 'politician', 'isasian': 'non-asian'}, {'name': 'david wu', 'identity': 'business owner', 'isasian': 'asian'}, {'name': 'don jerome', 'identity': 'politician', 'isasian': 'non-asian'}, {'name': 'grace chan mckibben', 'identity': 'business owner', 'isasian': 'asian'}]}, {'entities': [{'name': 'yin, alice', 'identity': 'journalist/media', 'isasian': 'asian'}, {'name': 'julie moy', 'identity': 'general public', 'isasian': 'asian'}, {'name': 'jason', 'identity': 'general public', 'isasian': 'asian'}, {'name': 'weizhong xiong', 'identity': 'victim', 'isasian': 'asian

In [17]:
output_file_path = "../../test/result.json"
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, indent=4)